# Portfolio optimization

This notebook is based on https://jump.dev/JuMP.jl/dev/tutorials/nonlinear/portfolio/, with new content for conic formulations.

In [ ]:
using JuMP
import Clarabel
import Statistics
import LinearAlgebra
import Dualization

Recall the [Markowitz model](https://en.wikipedia.org/wiki/Markowitz_model) for portfolio optimization.
The task is to choose a combination of assets to purchase with a fixed budget $B$.
We assume the future returns on $N$ assets follow a Gaussian distribution with known mean $r$ and covariance $\Sigma$.

The portfolio $x$ that minimizes risk subject to obtaining a desired expected level of returns $R$ is the solution of the optimization problem:

$$
\begin{align}
\min&\,x^T\Sigma x\\
s.t.&\sum_{i=1}^N x_i \le B \\
&r^Tx \ge R\\
&x \ge 0
\end{align}
$$

We'll use some very old data (from [here](https://www2.isye.gatech.edu/~sahmed/isye6669/)) to set up a toy version of this problem.

| Month        |  IBM     |  WMT    |  SEHI  |
|--------------|----------|---------|--------|
| November-00  |  93.043  |  51.826 |  1.063 |
| December-00  |  84.585  |  52.823 |  0.938 |
| January-01   |  111.453 |  56.477 |  1.000 |
| February-01  |  99.525  |  49.805 |  0.938 |
| March-01     |  95.819  |  50.287 |  1.438 |
| April-01     |  114.708 |  51.521 |  1.700 |
| May-01       |  111.515 |  51.531 |  2.540 |
| June-01      |  113.211 |  48.664 |  2.390 |
| July-01      |  104.942 |  55.744 |  3.120 |
| August-01    |  99.827  |  47.916 |  2.980 |
| September-01 |  91.607  |  49.438 |  1.900 |
| October-01   |  107.937 |  51.336 |  1.750 |
| November-01  |  115.590 |  55.081 |  1.800 |

In [ ]:
stock_data = [
    93.043 51.826 1.063
    84.585 52.823 0.938
    111.453 56.477 1.000
    99.525 49.805 0.938
    95.819 50.287 1.438
    114.708 51.521 1.700
    111.515 51.531 2.540
    113.211 48.664 2.390
    104.942 55.744 3.120
    99.827 47.916 2.980
    91.607 49.438 1.900
    107.937 51.336 1.750
    115.590 55.081 1.800
]

In [ ]:
stock_returns = (stock_data[2:13,:] - stock_data[1:12,:]) ./ stock_data[1:12,:]

In [ ]:
mean_returns = Statistics.mean(stock_returns; dims = 1)

In [ ]:
cov_returns = Statistics.cov(stock_returns)

Ok, time for some optimiztion!

In [ ]:
qp_portfolio = Model(Clarabel.Optimizer)
@variable(qp_portfolio, allocation[1:3] >= 0)
@objective(qp_portfolio, Min, allocation' * cov_returns * allocation)
@constraint(qp_portfolio, sum(allocation) <= 1000)
@constraint(qp_portfolio, sum(mean_returns[i] * allocation[i] for i in 1:3) >= 50)
optimize!(qp_portfolio)

In [ ]:
objective_value(qp_portfolio)

In [ ]:
value.(allocation)

## Conic version

You may have heard something about [conic optimization](https://en.wikipedia.org/wiki/Conic_optimization). We'll reformulate the above problem into the form of a conic optimization problem.

JuMP's [`SecondOrderCone()`](https://jump.dev/JuMP.jl/stable/reference/constraints/#JuMP.SecondOrderCone) is defined as the set
$\{(t, x) \in \mathbb{R}^{\text{dim}} : t \ge ||x||_2\}$.

Given a Cholesky decomposition $Q = LL^T$, it follows that $x^TQx = x^TLL^Tx = ||L^Tx||^2$. 

In [ ]:
cov_factor = Matrix(#= Compute the L or L^T factor of Q here =#)

In [ ]:
socp_portfolio = Model(Clarabel.Optimizer)
@variable(socp_portfolio, allocation[1:3] >= 0)
@variable(socp_portfolio, soc_epigraph)
@objective(socp_portfolio, Min, soc_epigraph)
@constraint(socp_portfolio, [soc_epigraph; #= fill in here =#] in SecondOrderCone())
@constraint(socp_portfolio, sum(allocation) <= 1000)
@constraint(socp_portfolio, sum(mean_returns[i] * allocation[i] for i in 1:3) >= 50)
optimize!(socp_portfolio)

In [ ]:
objective_value(socp_portfolio)^2

In [ ]:
value.(allocation)

In [ ]:
print(socp_portfolio)

In [ ]:
dualized_socp_portfolio = Dualization.dualize(socp_portfolio)
print(dualized_socp_portfolio)

In [ ]:
set_optimizer(socp_portfolio, Dualization.dual_optimizer(Clarabel.Optimizer))

In [ ]:
optimize!(socp_portfolio)

In [ ]:
objective_value(socp_portfolio)^2

In [ ]:
value.(allocation)